In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tqdm
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import requests
from nba_api.stats.static.players import find_players_by_full_name
from adjustText import adjust_text
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import time
from PIL import Image
from collections import defaultdict

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.leaguedashteamshotlocations import LeagueDashTeamShotLocations
from nba_api.stats.endpoints.leaguedashplayerptshot import LeagueDashPlayerPtShot

from nba_api.stats.endpoints.playerdashboardbyshootingsplits import PlayerDashboardByShootingSplits
from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats
from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats

from nba_api.stats.endpoints.leaguehustlestatsplayer import LeagueHustleStatsPlayer
from nba_api.stats.endpoints.leaguedashteamstats import LeagueDashTeamStats

In [4]:
harden_id = find_players_by_full_name("James Harden")[0]['id']

In [5]:
harden_shotchart = PlayerDashboardByShootingSplits(player_id=harden_id, season='2021-22')

In [6]:
harden_shotchart.get_data_frames()[5]

,GROUP_SET,GROUP_VALUE,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,EFG_PCT,BLKA,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,CFID,CFPARAMS
0,Shot Type Summary,Alley Oop,0,0,0.000,0,0,0.000,0.000,0,0.000,0.000,0.0,0.0,0.000,0.000,300,Alley Oop
1,Shot Type Summary,Bank Shot,4,7,0.571,0,0,0.000,0.571,0,0.000,1.000,0.0,0.0,0.000,1.000,300,Bank Shot
2,Shot Type Summary,Dunk,1,1,1.000,0,0,0.000,1.000,0,1.000,0.000,0.0,0.0,1.000,0.000,300,Dunk
3,Shot Type Summary,Fadeaway,1,1,1.000,1,1,1.000,1.500,0,0.000,0.000,0.0,1.0,0.000,1.000,300,Fadeaway
4,Shot Type Summary,Finger Roll,1,6,0.167,0,0,0.000,0.167,4,0.000,1.000,0.0,0.0,0.000,1.000,300,Finger Roll
5,Shot Type Summary,Hook Shot,0,1,0.000,0,0,0.000,0.000,0,0.000,0.000,0.0,0.0,0.000,0.000,300,Hook Shot
6,Shot Type Summary,Jump Shot,72,183,0.393,50,132,0.379,0.530,7,0.091,0.909,0.3,0.7,0.236,0.764,300,Jump Shot
7,Shot Type Summary,Layup,29,60,0.483,0,0,0.000,0.483,15,0.000,1.000,0.0,0.0,0.000,1.000,300,Layup
8,Shot Type Summary,Tip Shot,2,4,0.500,0,0,0.000,0.500,0,0.000,1.000,0.0,0.0,0.000,1.000,300,Tip Shot


In [7]:
# 3 lokacij
# 5 tip

In [8]:
seasons = [f"{item}-{str(item+1)[-2:]}" for item in range(2009, 2022)]

In [9]:
df_types = []
df_locs = []
for season in seasons:
    data = PlayerDashboardByShootingSplits(player_id=harden_id, season=season)
    loc_df = data.get_data_frames()[3]
    total_fga = loc_df.FGA.sum()
    loc_df.loc[:, 'SEASON'] = season
    loc_df.loc[:, 'LOC_PCT'] = loc_df.FGA / total_fga
    type_df = data.get_data_frames()[6]
    type_df.loc[:, 'SEASON'] = season
    type_df.loc[:, 'TYPE_PCT'] = type_df.FGA / total_fga
    df_types.append(type_df)
    df_locs.append(loc_df)
    time.sleep(0.25)

In [10]:
full_loc = pd.concat(df_locs)
full_type = pd.concat(df_types)

In [11]:
full_loc.loc[:, 'LOC_PCT'] = full_loc.LOC_PCT * 100
full_type.loc[:, 'TYPE_PCT'] = full_type.TYPE_PCT * 100

In [12]:
target_types = [
     'Jump Shot',
     'Layup Shot',
     'Driving Layup Shot',
     'Dunk Shot',
     'Pullup Jump shot',
     'Step Back Jump shot',
     'Floating Jump shot',
     'Driving Floating Jump Shot'
]

In [13]:
target_types = ['Jump Shot',
 'Layup Shot',
 'Driving Layup Shot',
 'Pullup Jump shot',
 'Floating Jump shot',
 'Step Back Jump shot',
 'Driving Floating Jump Shot',
 'Driving Finger Roll Layup Shot',
 'Driving Floating Bank Jump Shot',
 'Driving Reverse Layup Shot'
]

In [14]:
#target_types = []
reformatted = []
for season in seasons:
    cur_df = full_type.loc[full_type.SEASON == season]
    cur_df = cur_df.loc[cur_df.GROUP_VALUE.isin(target_types)]
    cur_df = cur_df.loc[:, ['SEASON', 'GROUP_VALUE', 'TYPE_PCT']]
    reformatted.append(cur_df)
    #top_df = cur_df.sort_values(by='FGA', ascending=False).iloc[:7]
    #top_types = top_df.GROUP_VALUE.unique().tolist()
    #for _t in top_types:
#        if _t not in target_types:
#            target_types.append(_t)

In [15]:
layup_types = [item for item in full_type.GROUP_VALUE.unique().tolist() if 'Layup' in item]

In [16]:
reformatted_df = pd.concat(reformatted)

In [17]:
pivoted_reformatted = reformatted_df.pivot_table(index='SEASON', columns='GROUP_VALUE', values='TYPE_PCT')

In [18]:
pivoted_reformatted = pivoted_reformatted.reset_index()


In [19]:
pivoted_reformatted = pivoted_reformatted.fillna(0.0)

In [20]:
pivoted_reformatted.loc[:, 'Driving Layup Shot'] = pivoted_reformatted['Driving Finger Roll Layup Shot'] + pivoted_reformatted['Driving Layup Shot']
pivoted_reformatted.loc[:, 'Driving Floating Jump Shot'] = pivoted_reformatted['Driving Floating Bank Jump Shot'] + pivoted_reformatted['Driving Floating Jump Shot']

In [21]:
pivoted_reformatted = pivoted_reformatted.loc[:, ['SEASON', 'Driving Floating Jump Shot', 'Driving Layup Shot', 'Jump Shot', 'Pullup Jump shot', 'Step Back Jump shot']]

In [22]:
pivoted_reformatted.round(2)

GROUP_VALUE,SEASON,Driving Floating Jump Shot,Driving Layup Shot,Jump Shot,Pullup Jump shot,Step Back Jump shot
0,2009-10,0.00,10.03,63.49,1.21,0.35
1,2010-11,0.00,11.11,61.11,2.19,2.19
2,2011-12,0.00,14.79,57.23,2.07,2.70
3,2012-13,0.00,17.43,48.17,5.46,5.24
4,2013-14,0.00,15.19,44.48,5.39,12.20
5,2014-15,0.00,20.27,41.16,4.49,12.99
6,2015-16,1.67,13.98,38.59,12.55,12.31
7,2016-17,3.40,16.85,42.13,9.41,10.12
8,2017-18,5.04,23.81,33.75,6.63,18.43
9,2018-19,10.32,21.11,18.28,7.86,30.75
